# あやめの分類問題

In [2]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

## データの収集

In [3]:
from sklearn.datasets import load_iris

iris = load_iris()

In [4]:
X = iris.data
y = iris.target

In [5]:
df = pd.DataFrame(X, columns=iris.feature_names)
df['y'] = y
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
df['y'].value_counts()

2    50
1    50
0    50
Name: y, dtype: int64

評価指標: 正答率 ( Accuracy )  
正答率 = 正解数 / データ数 * 100
評価プロトコル: ホールドアウト ( データをトレーニングとテストに分割 )

In [28]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(df.drop(columns=['y']), df['y'], train_size=0.7, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(df.drop(columns=['y']), df['y'], train_size=0.7, random_state=0)

print(X_trainval.shape, X_valid.shape)
print(y_trainval.shape, y_valid.shape)

(105, 4) (45, 4)
(105,) (45,)


In [29]:
np.bincount(y_trainval), np.bincount(y_valid)

(array([34, 32, 39], dtype=int64), array([16, 18, 11], dtype=int64))

## 予測モデル構築 ( ロジスティック回帰 )

In [32]:
from sklearn.linear_model import LogisticRegression

logistic_regression01 = LogisticRegression(C=1.0)

logistic_regression01.fit(X_trainval, y_trainval)

print(f'Train Acc: {logistic_regression01.score(X_train, y_train):.3f}')
print(f'Valid Acc: {logistic_regression01.score(X_trainval, y_trainval):.3f}')

Train Acc: 0.943
Valid Acc: 0.943


In [33]:
from sklearn.linear_model import LogisticRegression

logistic_regression02 = LogisticRegression(C=0.01)

logistic_regression02.fit(X_trainval, y_trainval)

print(f'Train Acc: {logistic_regression02.score(X_train, y_train):.3f}')
print(f'Valid Acc: {logistic_regression02.score(X_trainval, y_trainval):.3f}')

Train Acc: 0.695
Valid Acc: 0.695


In [18]:
from sklearn.linear_model import LogisticRegression

logistic_regression01 = LogisticRegression(C=100)

logistic_regression01.fit(X_trainval, y_trainval)

print(f'Train Acc: {logistic_regression01.score(X_trainval, y_trainval):.3f}')
print(f'Valid Acc: {logistic_regression01.score(X_valid, y_valid):.3f}')

Train Acc: 0.990
Valid Acc: 0.956


In [24]:
final_model = LogisticRegression(C=100)

final_model.fit(X_trainval, y_trainval)

print(final_model.score(X_trainval,y_trainval))

0.9904761904761905


In [26]:
print(f'Test Acc: {final_model.score(X_test,y_test)}')

NameError: name 'X_test' is not defined

In [26]:
X_new = np.array([
    [0.2, 0.03, 0.4, 0.5],
    [4.1, 1.5, 8.3, 2.8]
])


logistic_regression.predict(X_new)

array([2, 2])

入力に対し、3つのベクトル ( 確率 )を返す➡その中で最も高い値をとるものをこたえる

In [29]:
#　確率ベクトルを返す
logistic_regression.predict_proba(X_new)

array([[1.80968130e-01, 3.97349056e-01, 4.21682814e-01],
       [5.39641905e-08, 3.96397713e-01, 6.03602233e-01]])

## 正答率による評価

In [30]:
logistic_regression.score(X_train, y_train)

0.9428571428571428

In [31]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_train, y_pred=logistic_regression.predict(X_train))

0.9428571428571428

In [32]:
acc_train = logistic_regression.score(X_train, y_train)
acc_test = logistic_regression.score(X_test, y_test)

print(f'Train Acc: {acc_train: .3f}')
print(f'Test Acc: {acc_test: .3f}')

Train Acc:  0.943
Test Acc:  0.889


学習方法をより良いものにしようとチューニングしても必ずしも良くなるとは限らない  
→そのテストデータのみに特化した学習方法が完成してしまい、ほかのものに対応できない  

対策: データをさらに分割し、訓練、検証用のデータを作る  
→最後にテストデータで試し、精度を確認する  

- 訓練データ: モデルを学習させるため
- 検証データ: モデルを選択するため
- テストデータ: 最終評価をするため

おわり